<a href="https://colab.research.google.com/github/jihunkim625/201500844_JiHunKim/blob/main/%EB%A9%9C%EB%A1%A0_%EC%96%B8%EC%96%B4%EB%B6%84%EC%84%9D_%EC%8B%AC%ED%99%94_%EA%B9%80%EC%A7%80%ED%9B%88_201500844.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 환경 준비

In [1]:
# konlpy(한국어형태소분석기)를 실행하기
import konlpy

ModuleNotFoundError: ignored

In [2]:
# konlpy 설치하기 ## https://data1000.tistory.com/33
!pip3 install jpype1==0.7.0
!pip3 install konlpy

     |████████████████████████████████| 2.7MB 9.9MB/s 
     |████████████████████████████████| 19.4MB 246kB/s 
     |████████████████████████████████| 92kB 7.7MB/s 
  Found existing installation: tweepy 3.6.0
    Uninstalling tweepy-3.6.0:
      Successfully uninstalled tweepy-3.6.0
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [3]:
# konlpy를 실행하기
import konlpy

In [4]:
## pandas을 실행하기, 단 실행 별명을 pd로
import pandas as pd

# 데이터 불러오기

In [5]:
# 구글 드라이브 연결을 위한 기본 세팅 
!pip install -U -q PyDrive
 
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [39]:
## 문서 ID로 실제 파일 불러오기
# 실습을 위한 역대 대통령 취임사: 노무현, 이명박, 박근혜, 문재인
## https://drive.google.com/file/d/1kP4v2yKWMCisvrif34hGstfm9nme7GVC/view?usp=sharing

rawdata_downloaded = drive.CreateFile({'id': '1kP4v2yKWMCisvrif34hGstfm9nme7GVC'})
rawdata_downloaded.GetContentFile('rawdata.tsv')

In [44]:
# "rawdata.txt" 파일의 내용을 "원본데이터" 변수로 불러오기
원본데이터 = pd.read_csv('rawdata.tsv', sep="\t")

In [45]:
# "원본데이터" 변수 내용 확인하기
원본데이터

,ID,Year,Title,Singer,Lyric
0,1,1964,동백아가씨,이미자,헤일 수 없이 수많은 밤을 내 가슴 도려내는 아픔에 겨워 얼마나 울었던가 동백아가씨...
1,2,1964,마도로스 부기,백야성,항구의 일~~번지 부기우기 일~번지 그라스를 채워~다오 부기우기 아~~가씨 고동이 ...
2,3,1964,이국땅,남일해,하룻밤 뱃길이면 가고 오건만 못 가는 이국땅에 서러움이 많 아현해탄 파도 위에 비친...
3,4,1964,첫사랑 마도로스,남일해,푸르른 달빛이 파도에 부서지면 파이프에 꿈을 실은 첫사랑 마도로스 배귀에 기대서면 ...
4,5,1964,눈물의 연평도,최숙자,조기를 담뿍잡아 기폭을 올리고 온다던 그배는 어이하여 아니오나 수평선 바라보며 그이...
...,...,...,...,...,...
2271,2272,2020,둘만의 세상으로 가,Crush,바람 같은 그대가 나를 스쳐가네요 잠시 쉬어 날 바라봐요 이 모든 날들이 영원할 순...
2272,2273,2020,Black Swan,방탄소년단,Do your thang Do your thang with me now Do you...
2273,2274,2020,아마두,"염따, 딥플로우, 팔로알토, 더콰이엇",어 그래 얘들아 사는게 개같지? 내 얼굴도 개같지 뭐얌 근데도 부자가 됐다 세상은 ...
2274,2275,2020,HIP,마마무,All I wanna be is 멋짐 내 마음대로 골라 kick it 머리 어깨 무...


# 형태소 분석

In [46]:
# konlpy(한국어형태소분석기) 중에서 Okt 실행하기
from konlpy.tag import Okt
okt = Okt()

In [60]:
# "원본데이터"를 대상으로 "okt"를 사용해서 형태소 분석하기
저장공간  = []
for index, row in 원본데이터.iterrows(): # 원본데이터의 컬럼(열)을 불러와서 반복할 준비!
    대상텍스트 = row[4] # 분석에서 사용할 텍스트 정보가 있는 열을 지정해준다. 주의! python은 0부터 시작한다!
    분류 = row[1] # 분석에서 사용할 분류 정보가 있는 열을 지정해준다. 주의! python은 0부터 시작한다!
    형태소 = list(okt.pos(대상텍스트)) # okt의 pos 기능을 활용해서 대상 텍스트를 형태소 분리한다.
    형태소종합 = pd.DataFrame.from_records(형태소) # 형태소에 저장된 내용을 데이터프레임 형태로 해서 형태소종합으로 저장한다.
    형태소종합['type'] = 분류 # 형태소종합에 분류정보를 추가한다.
    형태소종합['count'] = 1   # 형태소종합에 카운트정보를 추가한다.   
    저장공간.append(형태소종합) # 형태소종합의 내용을 종합하여 저장한다.

In [61]:
# "형태소" 변수의 저장 내용 확인
형태소

[('All', 'Alpha'),
 ('the', 'Alpha'),
 ('colors', 'Alpha'),
 ('and', 'Alpha'),
 ('personalities', 'Alpha'),
 ('모든', 'Noun'),
 ('색깔', 'Noun'),
 ('과', 'Josa'),
 ('성격', 'Noun'),
 ('들', 'Suffix'),
 ('you', 'Alpha'),
 ('can', 'Alpha'),
 ('’', 'Punctuation'),
 ('t', 'Alpha'),
 ('see', 'Alpha'),
 ('right', 'Alpha'),
 ('through', 'Alpha'),
 ('what', 'Alpha'),
 ('I', 'Alpha'),
 ('truly', 'Alpha'),
 ('am', 'Alpha'),
 ('그것', 'Noun'),
 ('들', 'Suffix'),
 ('사이', 'Noun'),
 ('로는', 'Josa'),
 ('내', 'Noun'),
 ('가', 'Josa'),
 ('정말', 'Noun'),
 ('어떤', 'Adjective'),
 ('사람', 'Noun'),
 ('인지', 'Josa'),
 ('볼', 'Noun'),
 ('수', 'Noun'),
 ('없어', 'Adjective'),
 ('you', 'Alpha'),
 ('’', 'Punctuation'),
 ('re', 'Alpha'),
 ('hurting', 'Alpha'),
 ('me', 'Alpha'),
 ('without', 'Alpha'),
 ('noticing', 'Alpha'),
 ('넌', 'Noun'),
 ('예고', 'Noun'),
 ('도', 'Josa'),
 ('없이', 'Adverb'),
 ('나', 'Noun'),
 ('에게', 'Josa'),
 ('상처', 'Noun'),
 ('를', 'Josa'),
 ('주고', 'Verb'),
 ('I', 'Alpha'),
 ('’', 'Punctuation'),
 ('m', 'Alpha'),
 ('so'

In [62]:
# "형태소종합" 변수의 저장 내용 확인
형태소종합

,0,1,type,count
0,All,Alpha,2020,1
1,the,Alpha,2020,1
2,colors,Alpha,2020,1
3,and,Alpha,2020,1
4,personalities,Alpha,2020,1
...,...,...,...,...
793,의,Josa,2020,1
794,것,Noun,2020,1
795,이,Josa,2020,1
796,되어줄게,Verb,2020,1


In [63]:
# "저장공간" 변수의 저장 내용 확인
저장공간

[        0           1  type  count
 0      헤일        Noun  1964      1
 1       수        Noun  1964      1
 2      없이      Adverb  1964      1
 3     수많은   Adjective  1964      1
 4       밤        Noun  1964      1
 5       을        Josa  1964      1
 6       내        Noun  1964      1
 7      가슴        Noun  1964      1
 8    도려내는        Verb  1964      1
 9      아픔        Noun  1964      1
 10      에        Josa  1964      1
 11     겨워        Verb  1964      1
 12    얼마나        Noun  1964      1
 13   울었던가        Verb  1964      1
 14  동백아가씨        Noun  1964      1
 15    그리움        Noun  1964      1
 16      에        Josa  1964      1
 17    지쳐서        Verb  1964      1
 18     울다        Verb  1964      1
 19    지쳐서        Verb  1964      1
 20     꽃잎        Noun  1964      1
 21      은        Josa  1964      1
 22    빨갛게   Adjective  1964      1
 23      멍        Noun  1964      1
 24      이        Josa  1964      1
 25    들었오        Verb  1964      1
 26    동백꽃        Noun  1964

In [64]:
# "저장공간" 변수의 내용을 데이터프레임 형식으로 변환해서 "분석통합" 변수에 저장한다.
분석통합 = pd.concat(저장공간)

In [65]:
# "분석통합" 변수의 저장 내용 확인
분석통합

,0,1,type,count
0,헤일,Noun,1964,1
1,수,Noun,1964,1
2,없이,Adverb,1964,1
3,수많은,Adjective,1964,1
4,밤,Noun,1964,1
...,...,...,...,...
793,의,Josa,2020,1
794,것,Noun,2020,1
795,이,Josa,2020,1
796,되어줄게,Verb,2020,1


In [66]:
# "분석통합" 변수의 컬럼(열) 이름 변경
분석통합.columns = ["형태소", "품사", "분류", "카운트"]

In [67]:
# "분석통합" 변수의 저장 내용 확인
분석통합

,형태소,품사,분류,카운트
0,헤일,Noun,1964,1
1,수,Noun,1964,1
2,없이,Adverb,1964,1
3,수많은,Adjective,1964,1
4,밤,Noun,1964,1
...,...,...,...,...
793,의,Josa,2020,1
794,것,Noun,2020,1
795,이,Josa,2020,1
796,되어줄게,Verb,2020,1


In [68]:
# "분석통합" 변수의 내용을 형태소, 품사, 분류가 같은 것을 합치고, 카운트의 총합을 구해서 "그룹통합" 변수에 저장한다.
그룹통합 = 분석통합.groupby(['형태소', '품사', "분류"])['카운트'].sum()

In [69]:
# "그룹통합" 변수의 저장 내용 확인
그룹통합

형태소   품사           분류  
!     Punctuation  1966     2
                   1987    14
                   1992     4
                   1996     1
                   2001     1
                           ..
힙합씬   Noun         2019     1
힙힙    Noun         2006     1
힛죽햇죽  Noun         1983     4
？     Foreign      1978     2
�     Foreign      1994     1
Name: 카운트, Length: 101037, dtype: int64

In [70]:
# "그룹통합" 변수의 내용을 데이터프레임 형식으로 변환해서 "그룹통합" 변수에 저장한다.
그룹통합 = pd.DataFrame(그룹통합)

In [71]:
# "그룹통합" 변수의 저장 내용 확인
그룹통합

카운트
형태소  품사          분류       
!    Punctuation 1966    2
                 1987   14
                 1992    4
                 1996    1
                 2001    1
...                    ...
힙합씬  Noun        2019    1
힙힙   Noun        2006    1
힛죽햇죽 Noun        1983    4
？    Foreign     1978    2
�    Foreign     1994    1

[101037 rows x 1 columns]

In [72]:
# "그룹통합" 변수의 내용을 "형태소분석결과.csv" 파일로 저장한다.
# header는 컬럼(열) 정보의 포함 여부이다.
# encoding은 문자코드를 선택하는 것이다. python에서는 기본적으로 utf-8(유니코드)를 사용한다.

그룹통합.to_csv('형태소분석결과.csv', header='true', encoding='utf-8')